In [1]:
import tensorflow as tf

# Replace with your actual GCS bucket name
bucket_name = "creature-vision-training-set"

# Use recursive glob pattern
tfrecord_paths = tf.io.gfile.glob(f"gs://{bucket_name}/processed/**/*.tfrecord")

# Print results
if not tfrecord_paths:
    print(f"No TFRecord files found in bucket: {bucket_name}")
else:
    print(f"Found {len(tfrecord_paths)} TFRecord files:")
    for path in tfrecord_paths:
        print(path)


Found 24 TFRecord files:
gs://creature-vision-training-set/processed/v3_1/data-00000-of-00008.tfrecord
gs://creature-vision-training-set/processed/v3_1/data-00001-of-00008.tfrecord
gs://creature-vision-training-set/processed/v3_1/data-00002-of-00008.tfrecord
gs://creature-vision-training-set/processed/v3_1/data-00003-of-00008.tfrecord
gs://creature-vision-training-set/processed/v3_1/data-00004-of-00008.tfrecord
gs://creature-vision-training-set/processed/v3_1/data-00005-of-00008.tfrecord
gs://creature-vision-training-set/processed/v3_1/data-00006-of-00008.tfrecord
gs://creature-vision-training-set/processed/v3_1/data-00007-of-00008.tfrecord
gs://creature-vision-training-set/processed/v3_2/data-00000-of-00007.tfrecord
gs://creature-vision-training-set/processed/v3_2/data-00001-of-00007.tfrecord
gs://creature-vision-training-set/processed/v3_2/data-00002-of-00007.tfrecord
gs://creature-vision-training-set/processed/v3_2/data-00003-of-00007.tfrecord
gs://creature-vision-training-set/proce